In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/capstone-phase-1/checkpoint_labeled_200.csv
/kaggle/input/capstone-phase-1/jury_results.csv
/kaggle/input/capstone-phase-1/labeled_sample.csv
/kaggle/input/capstone-phase-1/jury_final_normalized.csv
/kaggle/input/capstone-phase-1/bcls.csv
/kaggle/input/capstone-phase-1/state.db
/kaggle/input/capstone-phase-1/checkpoint_labeled_400.csv
/kaggle/input/capstone-phase-1/checkpoint_asap_sample_20251103_184619_meta.json
/kaggle/input/capstone-phase-1/checkpoint_final_labeled_500_20251103_211458_meta.json
/kaggle/input/capstone-phase-1/jury_final.csv
/kaggle/input/capstone-phase-1/checkpoint_labeled_300.csv
/kaggle/input/capstone-phase-1/checkpoint_final_labeled_500_20251103_211458.pkl
/kaggle/input/capstone-phase-1/checkpoint_splits_SMALL.pkl
/kaggle/input/capstone-phase-1/checkpoint_splits_20251103_211458_meta.json
/kaggle/input/capstone-phase-1/checkpoint_splits_20251103_211458.pkl
/kaggle/input/capstone-phase-1/checkpoint_labeled_100.csv
/kaggle/input/capstone-phase-1/checkpo

In [7]:
# Replace this:
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# With this:
from transformers import AutoTokenizer, AutoModelForCausalLM
try:
    from transformers import BitsAndBytesConfig
    HAS_BNB = True
except Exception:
    HAS_BNB = False

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HUGGINGFACE_TOKEN")

In [5]:
!pip install -q bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 30.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.7 MB/s eta 0:00:00:00:0100:01
ERROR: pip's d

In [6]:
import pandas as pd
import numpy as np
import re
import json
import torch
import gc
from tqdm import tqdm
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from scipy.stats import spearmanr, mannwhitneyu, pearsonr
from sklearn.metrics import cohen_kappa_score
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter
import warnings
warnings.filterwarnings('ignore')

In [6]:
"""
BloomLLM-Jury: MEMORY-EFFICIENT Evaluation Script
==================================================
This version loads ONE model at a time to avoid GPU OOM errors.

Key difference from original:
- Load model 1 → Evaluate all essays → Unload → Clear memory
- Load model 2 → Evaluate all essays → Unload → Clear memory
- Load model 3 → Evaluate all essays → Unload → Clear memory
- Then aggregate results

This is slower but works with limited GPU memory.
"""

import pandas as pd
import numpy as np
import re
import json
import torch
import gc
from tqdm import tqdm
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from scipy.stats import spearmanr, mannwhitneyu, pearsonr
from sklearn.metrics import cohen_kappa_score
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================

CONFIG = {
    'models': [
        {
            'name': 'Phi',
            'base_id': 'microsoft/Phi-3-mini-4k-instruct',
            'lora_path': '/kaggle/working/models/phi3_lora'
        },
        {
            'name': 'Mistral',
            'base_id': 'mistralai/Mistral-7B-Instruct-v0.3',
            'lora_path': '/kaggle/working/models/mistral_lora'
        },
        {
            'name': 'Qwen',
            'base_id': 'Qwen/Qwen2.5-7B-Instruct',
            'lora_path': '/kaggle/working/models/qwen_lora'
        }
    ],
    'test_data_path': '/kaggle/working/checkpoint_splits_SMALL.pkl',  # UPDATED to use smaller set
    'output_dir': '/kaggle/working',
    'generation_params': {
        'max_new_tokens': 250,
        'do_sample': False,  # Greedy decoding for consistency
    },
}

# ASAP dataset score ranges per essay set (from official documentation)
ASAP_SCORE_RANGES = {
    1: (1, 6),    # Argumentative
    2: (1, 6),    # Argumentative
    3: (0, 3),    # Source-Dependent
    4: (0, 3),    # Source-Dependent
    5: (0, 4),    # Source-Dependent
    6: (0, 4),    # Source-Dependent
    7: (0, 3),    # Narrative
    8: (1, 6),    # Narrative
}

# ============================================================================
# MODEL LOADING (ONE AT A TIME)
# ============================================================================

def load_single_model(base_id, lora_path):
    """Load a single model with proper memory management."""
    print(f"\n{'='*80}")
    print(f"Loading {base_id}...")
    print(f"{'='*80}")
    
    # Clear any existing models
    torch.cuda.empty_cache()
    gc.collect()
    
    # Proper 4-bit quantization config
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )
    
    # Load base model
    base_model = AutoModelForCausalLM.from_pretrained(
        base_id,
        quantization_config=bnb_config,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
    )
    
    print(f"  ✓ Base model loaded")
    
    # Apply LoRA
    model = PeftModel.from_pretrained(base_model, lora_path)
    print(f"  ✓ LoRA adapter applied from {lora_path}")
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_id, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    print(f"  ✓ Tokenizer loaded")
    
    # Print memory usage
    if torch.cuda.is_available():
        mem_allocated = torch.cuda.memory_allocated(0) / 1e9
        mem_reserved = torch.cuda.memory_reserved(0) / 1e9
        print(f"  📊 GPU Memory: {mem_allocated:.2f}GB allocated, {mem_reserved:.2f}GB reserved")
    
    return model, tokenizer


def unload_model(model, tokenizer):
    """Properly unload model and free memory."""
    del model
    del tokenizer
    torch.cuda.empty_cache()
    gc.collect()
    print(f"  ✓ Model unloaded, memory cleared")


# ============================================================================
# SCORE EXTRACTION
# ============================================================================

def extract_score_robust(text):
    """Extract score and rationale from model output."""
    original_text = text
    
    # Clean up text
    assistant_markers = ['<|assistant|>', '[/INST]', 'assistant:', 'Assistant:']
    for marker in assistant_markers:
        if marker in text:
            text = text.split(marker)[-1]
    
    text = text.strip()
    
    # Method 1: JSON
    json_pattern = r'\{[^}]*"score"\s*:\s*\d+[^}]*\}'
    json_matches = re.findall(json_pattern, text, re.DOTALL | re.IGNORECASE)
    
    for match in json_matches:
        try:
            data = json.loads(match)
            score = int(data.get('score', 0))
            
            if 1 <= score <= 5:
                rationale = data.get('rationale', data.get('reason', data.get('explanation', '')))
                return score, rationale, "JSON"
        except:
            continue
    
    # Method 2: Patterns
    score_patterns = [
        r'"score"\s*:\s*(\d+)',
        r'score\s*:\s*(\d+)',
        r'Score\s*:\s*(\d+)',
        r'\*\*Score\*\*\s*:\s*(\d+)',
    ]
    
    for pattern in score_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            score = int(match.group(1))
            if 1 <= score <= 5:
                return score, "", "Pattern"
    
    # Method 3: Ratio
    ratio_patterns = [r'\b([1-5])\s*/\s*5\b', r'\b([1-5])\s+out\s+of\s+5\b']
    for pattern in ratio_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            return int(match.group(1)), "", "Ratio"
    
    # Method 4: Any digit 1-5
    digit_matches = re.findall(r'\b([1-5])\b', text)
    if digit_matches:
        return int(digit_matches[0]), "", "Weak"
    
    # Failed
    return 3, "Extraction failed", "Failed"


# ============================================================================
# INFERENCE
# ============================================================================

def construct_prompt(row):
    """Construct evaluation prompt."""
    system_msg = f"You are an expert educational evaluator specializing in Bloom's Taxonomy. You evaluate student responses for {row['bloom_level']}-level cognitive skills."
    
    user_msg = f"""Evaluate this student response for {row['bloom_level']}-level thinking according to Bloom's Taxonomy.

**Prompt**: {row['prompt']}

**Student Response**: {row['response']}

Provide your evaluation in this EXACT JSON format:
{{"score": <integer 1-5>, "rationale": "<brief explanation>"}}

**Scoring Guide**:
- 5: Excellent - Fully demonstrates {row['bloom_level']}-level thinking
- 4: Good - Strong demonstration with minor gaps
- 3: Adequate - Meets basic requirements
- 2: Weak - Minimal {row['bloom_level']}-level thinking
- 1: Poor - Does not demonstrate {row['bloom_level']}-level thinking

Respond ONLY with the JSON object."""

    return [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_msg}
    ]


def evaluate_with_model(model, tokenizer, row, config):
    """Evaluate a single essay with a model."""
    messages = construct_prompt(row)
    
    try:
        inputs = tokenizer.apply_chat_template(
            messages,
            return_tensors="pt",
            add_generation_prompt=True,
            padding=True,
            truncation=True
        ).to(model.device)
    except Exception as e:
        return {
            'score': 3,
            'rationale': f"Tokenization error: {e}",
            'extraction_method': 'Error',
            'raw_output': ''
        }
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_new_tokens=config['generation_params']['max_new_tokens'],
            do_sample=config['generation_params']['do_sample'],
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=False,  # Fix for DynamicCache error
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    score, rationale, method = extract_score_robust(response)
    
    return {
        'score': score,
        'rationale': rationale,
        'extraction_method': method,
        'raw_output': response
    }


# ============================================================================
# MAIN EVALUATION (ONE MODEL AT A TIME)
# ============================================================================

def evaluate_jury_memory_efficient(test_df, model_configs, config):
    """
    Evaluate jury by loading ONE model at a time.
    This avoids GPU OOM errors.
    """
    print("\n" + "="*80)
    print("MEMORY-EFFICIENT JURY EVALUATION")
    print("="*80)
    print("\nStrategy: Load → Evaluate → Unload → Repeat for each model")
    print("This is slower but works with limited GPU memory.\n")
    
    # Storage for all results
    all_model_results = {}  # {model_name: [scores for each essay]}
    all_rationales = {}     # {model_name: [rationales for each essay]}
    all_extraction_methods = {}  # {model_name: [methods for each essay]}
    
    # Evaluate with each model
    for model_config in model_configs:
        model_name = model_config['name']
        
        print(f"\n{'='*80}")
        print(f"EVALUATING WITH {model_name}")
        print(f"{'='*80}")
        
        # Load this model
        model, tokenizer = load_single_model(
            model_config['base_id'],
            model_config['lora_path']
        )
        
        # Evaluate all essays with this model
        model_scores = []
        model_rationales = []
        model_methods = []
        
        print(f"\nEvaluating {len(test_df)} essays...")
        
        for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc=f"{model_name}"):
            result = evaluate_with_model(model, tokenizer, row, config)
            
            model_scores.append(result['score'])
            model_rationales.append(result['rationale'])
            model_methods.append(result['extraction_method'])
        
        # Store results
        all_model_results[model_name] = model_scores
        all_rationales[model_name] = model_rationales
        all_extraction_methods[model_name] = model_methods
        
        # Print summary
        extraction_success = sum(1 for m in model_methods if m in ['JSON', 'Pattern']) / len(model_methods) * 100
        print(f"\n  📊 {model_name} Summary:")
        print(f"     Extraction success: {extraction_success:.1f}%")
        print(f"     Mean score: {np.mean(model_scores):.2f} ± {np.std(model_scores):.2f}")
        print(f"     Score distribution: {Counter(model_scores)}")
        
        # Unload this model before loading next
        unload_model(model, tokenizer)
    
    # Aggregate jury results
    print(f"\n{'='*80}")
    print("AGGREGATING JURY DECISIONS")
    print(f"{'='*80}")
    
    jury_scores = []
    for i in range(len(test_df)):
        essay_scores = [all_model_results[name][i] for name in all_model_results.keys()]
        jury_score = round(np.mean(essay_scores))
        jury_scores.append(jury_score)
    
    # Compile results
    results = {
        'individual_scores': all_model_results,
        'jury_scores': jury_scores,
        'rationales': [],
        'extraction_methods': all_extraction_methods,
        'true_scores': test_df['essay_score'].tolist(),
        'bloom_levels': test_df['bloom_level'].tolist(),
        'essay_sets': test_df['essay_set'].tolist() if 'essay_set' in test_df.columns else [None] * len(test_df),
    }
    
    # Format rationales
    for i in range(len(test_df)):
        essay_rationales = {name: all_rationales[name][i] for name in all_rationales.keys()}
        results['rationales'].append(essay_rationales)
    
    return results


# ============================================================================
# ANALYSIS FUNCTIONS (same as before)
# ============================================================================

def calculate_inter_jury_agreement(results, model_names):
    """Calculate Cohen's kappa between models."""
    print("\n" + "="*80)
    print("INTER-JURY AGREEMENT")
    print("="*80)
    
    pairwise_kappas = {}
    for i, name1 in enumerate(model_names):
        for j, name2 in enumerate(model_names):
            if i < j:
                kappa = cohen_kappa_score(
                    results['individual_scores'][name1],
                    results['individual_scores'][name2],
                    weights='linear'
                )
                pairwise_kappas[f"{name1}-{name2}"] = kappa
                print(f"  {name1:10s} vs {name2:10s}: κ = {kappa:.3f}")
    
    mean_kappa = np.mean(list(pairwise_kappas.values()))
    print(f"\n  📊 Mean κ: {mean_kappa:.3f}")
    
    return {'mean_kappa': mean_kappa, 'pairwise_kappas': pairwise_kappas}


def analyze_bloom_discrimination(results):
    """Test Bloom level discrimination."""
    print("\n" + "="*80)
    print("BLOOM'S LEVEL DISCRIMINATION")
    print("="*80)
    
    df = pd.DataFrame({
        'jury_score': results['jury_scores'],
        'bloom_level': results['bloom_levels']
    })
    
    print("\nScore Distribution by Bloom Level:")
    print(df.groupby('bloom_level')['jury_score'].describe())
    
    bloom_levels = df['bloom_level'].unique()
    
    if len(bloom_levels) == 2:
        level1, level2 = bloom_levels
        scores1 = df[df['bloom_level'] == level1]['jury_score']
        scores2 = df[df['bloom_level'] == level2]['jury_score']
        
        statistic, p_value = mannwhitneyu(scores1, scores2, alternative='two-sided')
        
        print(f"\n📊 Mann-Whitney U Test: {level1} vs {level2}")
        print(f"  p-value: {p_value:.4f}")
        print(f"  {'Significant!' if p_value < 0.05 else 'Not significant'}")
        
        return {'p_value': p_value, 'significant': p_value < 0.05}
    
    return None


def calculate_asap_correlation(results):
    """Calculate normalized correlation with ASAP."""
    print("\n" + "="*80)
    print("ASAP CORRELATION")
    print("="*80)
    
    jury_norm = [(s - 1) / 4 for s in results['jury_scores']]
    asap_norm = []
    
    for score, essay_set in zip(results['true_scores'], results['essay_sets']):
        if essay_set and essay_set in ASAP_SCORE_RANGES:
            min_s, max_s = ASAP_SCORE_RANGES[essay_set]
            norm = (score - min_s) / (max_s - min_s)
            asap_norm.append(max(0, min(1, norm)))
        else:
            asap_norm.append(score / max(results['true_scores']))
    
    rho, p = spearmanr(asap_norm, jury_norm)
    
    print(f"\n📊 Spearman ρ: {rho:.3f} (p={p:.4f})")
    print(f"  {'Significant!' if p < 0.05 else 'Not significant'}")
    
    return {'rho': rho, 'p_value': p}


def export_results(results, model_names, output_dir):
    """Export results to CSV."""
    print("\n" + "="*80)
    print("EXPORTING RESULTS")
    print("="*80)
    
    # Main results
    results_df = pd.DataFrame({
        'true_score': results['true_scores'],
        'jury_score': results['jury_scores'],
        'bloom_level': results['bloom_levels'],
        **{f'{name}_score': results['individual_scores'][name] for name in model_names}
    })
    
    results_df.to_csv(f"{output_dir}/jury_results_complete.csv", index=False)
    print(f"  ✓ Saved: jury_results_complete.csv")


# ============================================================================
# MAIN
# ============================================================================

def main():
    """Main execution."""
    print("\n" + "="*80)
    print("BloomLLM-Jury: Memory-Efficient Evaluation")
    print("="*80)
    
    # Load test data
    print("\n[1/5] Loading test data...")
    test_df = pd.read_pickle(CONFIG['test_data_path'])['test']
    print(f"  ✓ Loaded {len(test_df)} essays")
    
    # Evaluate jury (one model at a time)
    print("\n[2/5] Evaluating jury...")
    results = evaluate_jury_memory_efficient(test_df, CONFIG['models'], CONFIG)
    
    # Get model names
    model_names = [m['name'] for m in CONFIG['models']]
    
    # Calculate metrics
    print("\n[3/5] Calculating metrics...")
    agreement = calculate_inter_jury_agreement(results, model_names)
    bloom = analyze_bloom_discrimination(results)
    asap = calculate_asap_correlation(results)
    
    # Export
    print("\n[4/5] Exporting results...")
    export_results(results, model_names, CONFIG['output_dir'])
    
    # Summary
    print("\n" + "="*80)
    print("SUMMARY")
    print("="*80)
    print(f"  Inter-jury κ: {agreement['mean_kappa']:.3f}")
    if bloom:
        print(f"  Bloom p-value: {bloom['p_value']:.4f}")
    print(f"  ASAP ρ: {asap['rho']:.3f}")
    print("\n✓ Evaluation complete!")


if __name__ == "__main__":
    main()

BloomLLM-Jury with WandB Logging

Loading data...
✓ Loaded 50 essays

EVALUATING: Phi

Loading microsoft/Phi-3-mini-4k-instruct...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✓ Loaded


Phi: 100%|██████████| 50/50 [1:16:14<00:00, 91.48s/it] 



Phi Results:
  Extraction: 16.0%
  Mean score: 1.18 ± 0.62
  Distribution: Counter({1: 45, 2: 3, 4: 2})

EVALUATING: Mistral

Loading mistralai/Mistral-7B-Instruct-v0.3...


...

Qwen: 100%|██████████| 50/50 [3:38:54<00:00, 262.68s/it]  



Qwen Results:
  Extraction: 8.0%
  Mean score: 1.24 ± 0.81
  Distribution: Counter({1: 46, 4: 4})


wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")



AGGREGATING

METRICS:
  Phi vs Mistral: κ = 0.000
  Phi vs Qwen: κ = -0.081
  Mistral vs Qwen: κ = 0.000

  Mean κ: -0.027
  Bloom discrimination: p = 1.0000
  ASAP correlation: ρ = -0.165

EXPORTING...

✅ COMPLETE!
  Inter-jury κ: -0.027
  Bloom p-value: 1.0000
  ASAP ρ: -0.165

📊 View results: https://wandb.ai/mishralaavanya-svkm-s-narsee-monjee-institute-of-managem/bloomllm-jury/runs/5hjsgkp8


Mistral_extraction_success,▁
Mistral_mean_score,▁
Mistral_score,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Mistral_std_score,▁
Phi_extraction_success,▁
Phi_mean_score,▁
Phi_score,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁█▁▁▁▁▁▃▁▁▁▃▁▁▁▁▁▁▃▁▁
Phi_std_score,▁
Qwen_extraction_success,▁
Qwen_mean_score,▁
Qwen_score,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁██▁▁▁▁▁▁▁▁▁▁


## Corrected Ealuation

In [ ]:
"""
BloomLLM-Jury: MEMORY-EFFICIENT Evaluation Script WITH WANDB
=============================================================
YOUR WORKING CODE - ONLY wandb.log() calls added, nothing else changed
"""

import pandas as pd
import numpy as np
import re
import json
import torch
import gc
from tqdm import tqdm
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from scipy.stats import spearmanr, mannwhitneyu, pearsonr
from sklearn.metrics import cohen_kappa_score
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter
import warnings
warnings.filterwarnings('ignore')

import wandb  # ONLY NEW LINE

# ============================================================================
# CONFIGURATION
# ============================================================================

CONFIG = {
    'models': [
        # {
        #     'name': 'Phi',
        #     'base_id': 'microsoft/Phi-3-mini-4k-instruct',
        #     'lora_path': '/kaggle/input/capstone-phase-1/models/phi3_lora'  # YOUR WORKING MODELS
        # },
        {
            'name': 'Mistral',
            'base_id': 'mistralai/Mistral-7B-Instruct-v0.3',
            'lora_path': '/kaggle/input/capstone-phase-1/models/mistral_lora'
        },
        {
            'name': 'Qwen',
            'base_id': 'Qwen/Qwen2.5-7B-Instruct',
            'lora_path': '/kaggle/input/capstone-phase-1/models/qwen_lora'
        }
    ],
    'test_data_path': '/kaggle/input/capstone-phase-1/checkpoint_splits_SMALL.pkl',
    'output_dir': '/kaggle/working',
    'generation_params': {
        'max_new_tokens': 250,
        'do_sample': False,
    },
}

ASAP_SCORE_RANGES = {
    1: (1, 6), 2: (1, 6), 3: (0, 3), 4: (0, 3),
    5: (0, 4), 6: (0, 4), 7: (0, 3), 8: (1, 6),
}

# ============================================================================
# MODEL LOADING (ONE AT A TIME) - UNCHANGED
# ============================================================================

def load_single_model(base_id, lora_path):
    """Load a single model with proper memory management."""
    print(f"\n{'='*80}")
    print(f"Loading {base_id}...")
    print(f"{'='*80}")
    
    torch.cuda.empty_cache()
    gc.collect()
    
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )
    
    base_model = AutoModelForCausalLM.from_pretrained(
        base_id,
        quantization_config=bnb_config,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
    )
    
    print(f"  ✓ Base model loaded")
    
    model = PeftModel.from_pretrained(base_model, lora_path)
    print(f"  ✓ LoRA adapter applied from {lora_path}")
    
    tokenizer = AutoTokenizer.from_pretrained(base_id, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    print(f"  ✓ Tokenizer loaded")
    
    if torch.cuda.is_available():
        mem_allocated = torch.cuda.memory_allocated(0) / 1e9
        mem_reserved = torch.cuda.memory_reserved(0) / 1e9
        print(f"  📊 GPU Memory: {mem_allocated:.2f}GB allocated, {mem_reserved:.2f}GB reserved")
        
        # WANDB LOG
        wandb.log({
            f"{base_id.split('/')[-1]}_gpu_allocated": mem_allocated,
            f"{base_id.split('/')[-1]}_gpu_reserved": mem_reserved
        })
    
    return model, tokenizer


def unload_model(model, tokenizer):
    """Properly unload model and free memory."""
    del model
    del tokenizer
    torch.cuda.empty_cache()
    gc.collect()
    print(f"  ✓ Model unloaded, memory cleared")


# ============================================================================
# SCORE EXTRACTION - UNCHANGED
# ============================================================================

def extract_score_robust(text):
    """Extract score and rationale from model output."""
    original_text = text
    
    assistant_markers = ['<|assistant|>', '[/INST]', 'assistant:', 'Assistant:']
    for marker in assistant_markers:
        if marker in text:
            text = text.split(marker)[-1]
    
    text = text.strip()
    
    # Method 1: JSON
    json_pattern = r'\{[^}]*"score"\s*:\s*\d+[^}]*\}'
    json_matches = re.findall(json_pattern, text, re.DOTALL | re.IGNORECASE)
    
    for match in json_matches:
        try:
            data = json.loads(match)
            score = int(data.get('score', 0))
            
            if 1 <= score <= 5:
                rationale = data.get('rationale', data.get('reason', data.get('explanation', '')))
                return score, rationale, "JSON"
        except:
            continue
    
    # Method 2: Patterns
    score_patterns = [
        r'"score"\s*:\s*(\d+)',
        r'score\s*:\s*(\d+)',
        r'Score\s*:\s*(\d+)',
        r'\*\*Score\*\*\s*:\s*(\d+)',
    ]
    
    for pattern in score_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            score = int(match.group(1))
            if 1 <= score <= 5:
                return score, "", "Pattern"
    
    # Method 3: Ratio
    ratio_patterns = [r'\b([1-5])\s*/\s*5\b', r'\b([1-5])\s+out\s+of\s+5\b']
    for pattern in ratio_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            return int(match.group(1)), "", "Ratio"
    
    # Method 4: Any digit 1-5
    digit_matches = re.findall(r'\b([1-5])\b', text)
    if digit_matches:
        return int(digit_matches[0]), "", "Weak"
    
    return 3, "Extraction failed", "Failed"


# ============================================================================
# INFERENCE - UNCHANGED
# ============================================================================

def construct_prompt(row):
    """Construct evaluation prompt."""
    system_msg = f"You are an expert educational evaluator specializing in Bloom's Taxonomy. You evaluate student responses for {row['bloom_level']}-level cognitive skills."
    
    user_msg = f"""Evaluate this student response for {row['bloom_level']}-level thinking according to Bloom's Taxonomy.

**Prompt**: {row['prompt']}

**Student Response**: {row['response']}

Provide your evaluation in this EXACT JSON format:
{{"score": <integer 1-5>, "rationale": "<brief explanation>"}}

**Scoring Guide**:
- 5: Excellent - Fully demonstrates {row['bloom_level']}-level thinking
- 4: Good - Strong demonstration with minor gaps
- 3: Adequate - Meets basic requirements
- 2: Weak - Minimal {row['bloom_level']}-level thinking
- 1: Poor - Does not demonstrate {row['bloom_level']}-level thinking

Respond ONLY with the JSON object."""

    return [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_msg}
    ]


def evaluate_with_model(model, tokenizer, row, config):
    """Evaluate a single essay with a model."""
    messages = construct_prompt(row)
    
    try:
        inputs = tokenizer.apply_chat_template(
            messages,
            return_tensors="pt",
            add_generation_prompt=True,
            padding=True,
            truncation=True
        ).to(model.device)
    except Exception as e:
        return {
            'score': 3,
            'rationale': f"Tokenization error: {e}",
            'extraction_method': 'Error',
            'raw_output': ''
        }
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_new_tokens=config['generation_params']['max_new_tokens'],
            do_sample=config['generation_params']['do_sample'],
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=False,
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    score, rationale, method = extract_score_robust(response)
    
    return {
        'score': score,
        'rationale': rationale,
        'extraction_method': method,
        'raw_output': response
    }


# ============================================================================
# MAIN EVALUATION (ONE MODEL AT A TIME) - WANDB ADDED
# ============================================================================

def evaluate_jury_memory_efficient(test_df, model_configs, config):
    """Evaluate jury by loading ONE model at a time."""
    print("\n" + "="*80)
    print("MEMORY-EFFICIENT JURY EVALUATION")
    print("="*80)
    print("\nStrategy: Load → Evaluate → Unload → Repeat for each model")
    print("This is slower but works with limited GPU memory.\n")
    
    all_model_results = {}
    all_rationales = {}
    all_extraction_methods = {}
    
    for model_config in model_configs:
        model_name = model_config['name']
        
        print(f"\n{'='*80}")
        print(f"EVALUATING WITH {model_name}")
        print(f"{'='*80}")
        
        model, tokenizer = load_single_model(
            model_config['base_id'],
            model_config['lora_path']
        )
        
        model_scores = []
        model_rationales = []
        model_methods = []
        
        print(f"\nEvaluating {len(test_df)} essays...")
        
        for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc=f"{model_name}"):
            result = evaluate_with_model(model, tokenizer, row, config)
            
            model_scores.append(result['score'])
            model_rationales.append(result['rationale'])
            model_methods.append(result['extraction_method'])
        
        all_model_results[model_name] = model_scores
        all_rationales[model_name] = model_rationales
        all_extraction_methods[model_name] = model_methods
        
        extraction_success = sum(1 for m in model_methods if m in ['JSON', 'Pattern']) / len(model_methods) * 100
        mean_score = np.mean(model_scores)
        std_score = np.std(model_scores)
        
        print(f"\n  📊 {model_name} Summary:")
        print(f"     Extraction success: {extraction_success:.1f}%")
        print(f"     Mean score: {mean_score:.2f} ± {std_score:.2f}")
        print(f"     Score distribution: {Counter(model_scores)}")

        
        import json
        with open(f'/kaggle/working/{model_name}_outputs.json', 'w') as f:
            json.dump([
                {'essay_idx': i, 'score': model_scores[i], 'method': model_methods[i], 'rationale': model_rationales[i]} 
                for i in range(len(model_scores))
            ], f, indent=2)
        print(f"  ✓ Saved raw outputs to {model_name}_outputs.json")
        
        
        # WANDB LOG
        wandb.log({
            f"{model_name}_extraction_success": extraction_success,
            f"{model_name}_mean_score": mean_score,
            f"{model_name}_std_score": std_score,
        })
        wandb.log({f"{model_name}_score_dist": wandb.Histogram(model_scores)})
        
        unload_model(model, tokenizer)
    
    print(f"\n{'='*80}")
    print("AGGREGATING JURY DECISIONS")
    print(f"{'='*80}")
    
    jury_scores = []
    for i in range(len(test_df)):
        essay_scores = [all_model_results[name][i] for name in all_model_results.keys()]
        jury_scores.append(round(np.mean(essay_scores)))
    
    results = {
        'individual_scores': all_model_results,
        'jury_scores': jury_scores,
        'rationales': [],
        'extraction_methods': all_extraction_methods,
        'true_scores': test_df['essay_score'].tolist(),
        'bloom_levels': test_df['bloom_level'].tolist(),
        'essay_sets': test_df['essay_set'].tolist() if 'essay_set' in test_df.columns else [None] * len(test_df),
    }
    
    for i in range(len(test_df)):
        essay_rationales = {name: all_rationales[name][i] for name in all_rationales.keys()}
        results['rationales'].append(essay_rationales)
    
    return results


# ============================================================================
# ANALYSIS FUNCTIONS - WANDB ADDED
# ============================================================================

def calculate_inter_jury_agreement(results, model_names):
    """Calculate Cohen's kappa between models."""
    print("\n" + "="*80)
    print("INTER-JURY AGREEMENT")
    print("="*80)
    
    pairwise_kappas = {}
    for i, name1 in enumerate(model_names):
        for j, name2 in enumerate(model_names):
            if i < j:
                kappa = cohen_kappa_score(
                    results['individual_scores'][name1],
                    results['individual_scores'][name2],
                    weights='linear'
                )
                pairwise_kappas[f"{name1}-{name2}"] = kappa
                print(f"  {name1:10s} vs {name2:10s}: κ = {kappa:.3f}")
                
                # WANDB LOG
                wandb.log({f"kappa_{name1}_vs_{name2}": kappa})
    
    mean_kappa = np.mean(list(pairwise_kappas.values()))
    print(f"\n  📊 Mean κ: {mean_kappa:.3f}")
    
    # WANDB LOG
    wandb.log({"mean_kappa": mean_kappa})
    wandb.summary["mean_kappa"] = mean_kappa
    
    return {'mean_kappa': mean_kappa, 'pairwise_kappas': pairwise_kappas}


def analyze_bloom_discrimination(results):
    """Test Bloom level discrimination."""
    print("\n" + "="*80)
    print("BLOOM'S LEVEL DISCRIMINATION")
    print("="*80)
    
    df = pd.DataFrame({
        'jury_score': results['jury_scores'],
        'bloom_level': results['bloom_levels']
    })
    
    print("\nScore Distribution by Bloom Level:")
    print(df.groupby('bloom_level')['jury_score'].describe())
    
    bloom_levels = df['bloom_level'].unique()
    
    if len(bloom_levels) == 2:
        level1, level2 = bloom_levels
        scores1 = df[df['bloom_level'] == level1]['jury_score']
        scores2 = df[df['bloom_level'] == level2]['jury_score']
        
        statistic, p_value = mannwhitneyu(scores1, scores2, alternative='two-sided')
        
        print(f"\n📊 Mann-Whitney U Test: {level1} vs {level2}")
        print(f"  p-value: {p_value:.4f}")
        print(f"  {'Significant!' if p_value < 0.05 else 'Not significant'}")
        
        # WANDB LOG
        wandb.log({"bloom_p_value": p_value, "bloom_significant": p_value < 0.05})
        wandb.summary["bloom_p_value"] = p_value
        
        return {'p_value': p_value, 'significant': p_value < 0.05}
    
    return None


def calculate_asap_correlation(results):
    """Calculate normalized correlation with ASAP."""
    print("\n" + "="*80)
    print("ASAP CORRELATION")
    print("="*80)
    
    jury_norm = [(s - 1) / 4 for s in results['jury_scores']]
    asap_norm = []
    
    for score, essay_set in zip(results['true_scores'], results['essay_sets']):
        if essay_set and essay_set in ASAP_SCORE_RANGES:
            min_s, max_s = ASAP_SCORE_RANGES[essay_set]
            norm = (score - min_s) / (max_s - min_s)
            asap_norm.append(max(0, min(1, norm)))
        else:
            asap_norm.append(score / max(results['true_scores']))
    
    rho, p = spearmanr(asap_norm, jury_norm)
    
    print(f"\n📊 Spearman ρ: {rho:.3f} (p={p:.4f})")
    print(f"  {'Significant!' if p < 0.05 else 'Not significant'}")
    
    # WANDB LOG
    wandb.log({"asap_rho": rho, "asap_p_value": p})
    wandb.summary["asap_rho"] = rho
    
    return {'rho': rho, 'p_value': p}


def export_results(results, model_names, output_dir):
    """Export results to CSV."""
    print("\n" + "="*80)
    print("EXPORTING RESULTS")
    print("="*80)
    
    results_df = pd.DataFrame({
        'true_score': results['true_scores'],
        'jury_score': results['jury_scores'],
        'bloom_level': results['bloom_levels'],
        **{f'{name}_score': results['individual_scores'][name] for name in model_names}
    })
    
    csv_path = f"{output_dir}/jury_results_complete.csv"
    results_df.to_csv(csv_path, index=False)
    print(f"  ✓ Saved: jury_results_complete.csv")
    
    # WANDB LOG
    wandb.save(csv_path)
    wandb.log({"results_table": wandb.Table(dataframe=results_df)})


# ============================================================================
# MAIN - WANDB INIT ADDED
# ============================================================================

def main():
    """Main execution."""
    
    # WANDB INIT
    wandb.init(
        project="bloomllm-jury",
        name="jury-evaluation",
        id="uzfietg2",
        resume="allow",
        config=CONFIG
    )
    
    print("\n" + "="*80)
    print("BloomLLM-Jury: Memory-Efficient Evaluation")
    print("="*80)
    
    print("\n[1/5] Loading test data...")
    test_df = pd.read_pickle(CONFIG['test_data_path'])['test']
    print(f"  ✓ Loaded {len(test_df)} essays")
    
    wandb.log({"n_essays": len(test_df)})  # WANDB LOG
    
    print("\n[2/5] Evaluating jury...")
    results = evaluate_jury_memory_efficient(test_df, CONFIG['models'], CONFIG)
    
    model_names = [m['name'] for m in CONFIG['models']]
    
    print("\n[3/5] Calculating metrics...")
    agreement = calculate_inter_jury_agreement(results, model_names)
    bloom = analyze_bloom_discrimination(results)
    asap = calculate_asap_correlation(results)
    
    print("\n[4/5] Exporting results...")
    export_results(results, model_names, CONFIG['output_dir'])
    
    print("\n" + "="*80)
    print("SUMMARY")
    print("="*80)
    print(f"  Inter-jury κ: {agreement['mean_kappa']:.3f}")
    if bloom:
        print(f"  Bloom p-value: {bloom['p_value']:.4f}")
    print(f"  ASAP ρ: {asap['rho']:.3f}")
    print(f"\n📊 View results: {wandb.run.url}")  # WANDB LOG
    print("\n✓ Evaluation complete!")
    
    wandb.finish()  # WANDB FINISH


if __name__ == "__main__":
    main()

Mistral-7B-Instruct-v0.3_gpu_allocated,▁
Mistral-7B-Instruct-v0.3_gpu_reserved,▁
n_essays,▁
Mistral-7B-Instruct-v0.3_gpu_allocated,8.38205
Mistral-7B-Instruct-v0.3_gpu_reserved,9.11003
Phi-3-mini-4k-instruct_gpu_allocated,2.27042
Phi-3-mini-4k-instruct_gpu_reserved,3.79375
Phi_extraction_success,100
Phi_mean_score,3.38
Phi_std_score,0.95687
n_essays,50



BloomLLM-Jury: Memory-Efficient Evaluation

[1/5] Loading test data...
  ✓ Loaded 50 essays

[2/5] Evaluating jury...

MEMORY-EFFICIENT JURY EVALUATION

Strategy: Load → Evaluate → Unload → Repeat for each model
This is slower but works with limited GPU memory.


EVALUATING WITH Mistral

Loading mistralai/Mistral-7B-Instruct-v0.3...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

  ✓ Base model loaded
  ✓ LoRA adapter applied from /kaggle/input/capstone-phase-1/models/mistral_lora
  ✓ Tokenizer loaded
  📊 GPU Memory: 8.39GB allocated, 9.11GB reserved

Evaluating 50 essays...


Mistral: 100%|██████████| 50/50 [2:02:32<00:00, 147.05s/it]  


...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

  ✓ Tokenizer loaded
  📊 GPU Memory: 13.98GB allocated, 14.35GB reserved

Evaluating 50 essays...


Qwen:  60%|██████    | 30/50 [3:10:18<2:06:36, 379.82s/it]

In [6]:
# Quick check of what wandb logged
import wandb
api = wandb.Api()
run = api.run("mishralaavanya-svkm-s-narsee-monjee-institute-of-managem/bloomllm-jury/uzfietg2")

# Get all metrics
history = run.history()
print(history[['Phi_extraction_success', 'Phi_mean_score', 'Phi_std_score']])

   Phi_extraction_success  Phi_mean_score  Phi_std_score
0                     NaN             NaN            NaN
1                     NaN             NaN            NaN
2                   100.0            3.38        0.95687
3                     NaN             NaN            NaN
4                     NaN             NaN            NaN


In [4]:
!pip install transformers==4.38.0 --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 86.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 88.1 MB/s eta 0:00:00:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviou

In [1]:
# Restart runtime first, then run:
!pip uninstall numpy pandas scipy scikit-learn matplotlib seaborn -y
!pip uninstall transformers bitsandbytes accelerate peft -y

# Install compatible versions together
!pip install numpy==1.24.3
!pip install pandas scipy scikit-learn matplotlib seaborn
!pip install torch==2.1.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install transformers==4.36.2 bitsandbytes==0.41.3 accelerate==0.25.0 peft==0.7.1

Found existing installation: numpy 1.24.3
Uninstalling numpy-1.24.3:
  Successfully uninstalled numpy-1.24.3
Found existing installation: pandas 2.2.3
Uninstalling pandas-2.2.3:
  Successfully uninstalled pandas-2.2.3
Found existing installation: scipy 1.15.3
Uninstalling scipy-1.15.3:
  Successfully uninstalled scipy-1.15.3
Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2
Found existing installation: matplotlib 3.7.2
Uninstalling matplotlib-3.7.2:
  Successfully uninstalled matplotlib-3.7.2
Found existing installation: seaborn 0.12.2
Uninstalling seaborn-0.12.2:
  Successfully uninstalled seaborn-0.12.2
Found existing installation: transformers 4.36.0
Uninstalling transformers-4.36.0:
  Successfully uninstalled transformers-4.36.0
Found existing installation: bitsandbytes 0.41.0
Uninstalling bitsandbytes-0.41.0:
  Successfully uninstalled bitsandbytes-0.41.0
Found existing installation: accelerate 0.25.0
Uni

In [1]:
# Restart runtime, then:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install transformers==4.36.2 bitsandbytes==0.41.3 accelerate==0.25.0 peft==0.10.0

Looking in indexes: https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 1.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 4.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 71.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 11.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 23.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 11.0 MB/s eta 0:00:000

In [9]:
  import torch
  torch.cuda.empty_cache()

In [ ]:
import os
os._exit(00)

In [1]:
!pip install -q accelerate sentencepiece wandb tqdm scipy scikit-learn matplotlib seaborn

In [6]:
import transformers
import bitsandbytes
import peft
import torch

print(f"transformers: {transformers.__version__}")
print(f"bitsandbytes: {bitsandbytes.__version__}")
print(f"peft: {peft.__version__}")
print(f"torch: {torch.__version__}")

transformers: 4.41.2
bitsandbytes: 0.48.2
peft: 0.11.0
torch: 2.6.0+cu124


In [17]:
# Source - https://stackoverflow.com/questions/79786866/remoteentrynotfounderror-with-downloading-models-from-hugging-face-in-kaggle
# Posted by ANURAG SINGH BHANDARI
# Retrieved 5/11/2025, License - CC-BY-SA 4.0

!pip install -U transformers huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 109.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 83.4 MB/s eta 0:00:00:00:01
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviou

In [19]:
!pip install -U bitsandbytes

In [3]:
import torch
# Verify P100
print("="*60)
print("GPU CHECK")
print("="*60)
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    total_mem = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"✓ GPU: {gpu_name}")
    print(f"✓ Memory: {total_mem:.2f} GB")
    
    if "P100" not in gpu_name:
        print("⚠️ WARNING: Not using P100! Switch accelerator.")
    else:
        print("✓ Perfect! P100 is ideal for this task.")
else:
    print("✗ No GPU found!")
print("="*60)

GPU CHECK
✓ GPU: Tesla P100-PCIE-16GB
✓ Memory: 15.89 GB
✓ Perfect! P100 is ideal for this task.


In [1]:
import wandb
from kaggle_secrets import UserSecretsClient

# Get W&B key from Kaggle secrets
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")

# Login programmatically (no prompt)
wandb.login(key=wandb_api_key)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

True

In [7]:
print('hi')

hi
